# Regression: Simple Linear and multiple Regressions

In this notebook we will use data on house sales in King County that was sourced from Kaggle to predict house prices using simple (one input) linear regression. You will:
* Use pandas functions to compute important summary statistics
* Write a function to compute the Simple Linear Regression weights using the closed form solution
* Write a function to make predictions of the output given the input feature
* Compare two different models for predicting house prices

In this notebook you will be provided with some already complete code as well as some code that you should complete yourself in order to answer quiz questions. The code we provide to complete is optional and is there to assist you with solving the problems but feel free to ignore the helper code and write your own.

# Import libraries

In [ ]:
#importing numpy and pandas, seaborn

import numpy as np #linear algebra
import pandas as pd #data preprocessing, CSV file I/O
import seaborn as sns #for plotting graphs
import matplotlib.pyplot as plt
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn import linear_model

In [ ]:
#connect Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [ ]:
#Loading Data
sales = pd.read_csv('/content/drive/MyDrive/kc_house_data.csv')

In [ ]:
sales.shape #it should be (21613, 21)

(21613, 21)

In [ ]:
sales.head() #explore the data

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


# Split data into training and testing
We use random state = 0 so that everyone running this notebook gets the same results.  In practice, you may set a random state. 

In [ ]:
train_data,test_data = train_test_split(sales,train_size=0.8,random_state=0)

# Useful pandas/numpy summary functions

In order to make use of the closed form solution as well as take advantage of pandas' built in functions we will review some important ones. In particular:
* Computing the sum of a column/array (in a dataframe)
* Computing the arithmetic average (mean) of a column/array
* multiplying columns/arrays by constants
* multiplying columns/arrays by other columns/arrays

In [ ]:
# Let's compute the mean of the House Prices in King County in 2 different ways.
prices = sales['price'] # extract the price column of the sales dataframe -- this is now a column
# recall that the arithmetic average (the mean) is the sum of the prices divided by the total number of houses:
sum_prices = prices.sum()
num_houses = len(prices) # returns its length
avg_price_1 = sum_prices/num_houses
avg_price_2 = prices.mean() # if you just want the average, the .mean() function
print ("average price via method 1: " + str(avg_price_1))
print ("average price via method 2: " + str(avg_price_2))

average price via method 1: 540088.1417665294
average price via method 2: 540088.1417665294


You should the same answer both ways which is 540088

In [ ]:
# if we want to multiply every price by 0.5 it's a simple as:
half_prices = 0.5*prices
# Let's compute the sum of squares of price. We can multiply two arrays of the same length elementwise also with *
prices_squared = prices*prices
sum_prices_squared = prices_squared.sum() # price_squared is an array of the squares and we want to add them up.
print ("the sum of price squared is: " + str(sum_prices_squared)) #the sum of price squared is: 9217325138472070.0

the sum of price squared is: 9217325138472070.0


Aside: The python notation x.xxe+yy means x.xx \* 10^(yy). e.g 100 = 10^2 = 1*10^2 = 1e2 

# Build a generic simple linear regression function 

Armed with these Array functions we can use the closed form solution found from lecture to compute the slope and intercept for a simple linear regression on observations stored as Arrays: input_feature, output.

Complete the following function (or write your own) to compute the simple linear regression slope and intercept:

In [ ]:
def simple_linear_regression(input_feature, output):
  
    # compute the sum of input_feature and output
    n = len(input_feature)
    x = input_feature
    y = output
    
    # compute the mean of input_feature and output
    x_mean = x.mean() # x 的平均
    y_mean = y.mean() # y 的平均
    
    # compute the product of the output and the input_feature and its mean
    sum_xy = (x*y).sum() # 成對的 x & y 相乘後加總
    xy_by_n = (x.sum()*y.sum())/n # sum of x 乘上 sum of y 後求平均
               
    # compute the squared value of the input_feature and its mean
    x_square = (x**2).sum() # x^2 後的總和
    xx_by_n = (x.sum()**2)/n # sum of x 乘上 sum of x 後求平均
               
    # use the formula for the slope
    slope =  (sum_xy - xy_by_n)/(x_square - xx_by_n)
    
    # use the formula for the intercept
    intercept = y_mean - (slope * x_mean)

    return (intercept, slope)

We can test that our function works by passing it something where we know the answer. In particular we can generate a feature and then put the output exactly on a line: output = 1 + 1\*input_feature then we know both our slope and intercept should be 1

In [ ]:
# this is to test whether simple_linear_regression work
test_feature = np.array(range(5))
test_output = np.array(1 + 1*test_feature)
(test_intercept, test_slope) =  simple_linear_regression(test_feature, test_output)
print ("Intercept: " + str(test_intercept)) #it should be 1
print ("Slope: " + str(test_slope)) #it should be 1

Intercept: 1.0
Slope: 1.0


Now that we know it works let's build a regression model for predicting price based on sqft_living. Rembember that we train on train_data!

In [ ]:
sqft_intercept, sqft_slope = simple_linear_regression(train_data['sqft_living'], train_data['price'])

print ("Intercept: " + str(sqft_intercept))  # Intercept: -48257.063591028564
print ("Slope: " + str(sqft_slope)) # Slope: 283.96855715512993

Intercept: -48257.063591028564
Slope: 283.96855715512993


# Predicting Values

Now that we have the model parameters: intercept & slope we can make predictions. Complete the following function to return the predicted output given the input_feature, slope and intercept:

In [ ]:
def get_regression_predictions(input_feature, intercept, slope):
    # calculate the predicted values:
    predicted_values =  intercept + slope * input_feature # 已經知道 slope w1 & intercept w0, 對於某一輸入的input_feature 求得其估計值
    # y = f(x) = w0 +w1*x , w0 = Y-intercept, w1 = slope, y = predicted values
    return predicted_values

Now that we can calculate a prediction given the slope and intercept let's make a prediction. Use (or alter) the following to find out the estimated price for a house with 2650 squarefeet according to the squarefeet model we estiamted above.

**Quiz Question: Using your Slope and Intercept from (4), What is the predicted price for a house with 2650 sqft?**

In [ ]:
my_house_sqft = 2650 # 特徵（房子的坪數） 
estimated_price = get_regression_predictions(my_house_sqft, sqft_intercept, sqft_slope)# Using get_regression_predictions to find out
print ("The estimated price for a house with %d squarefeet is $%.2f" % (my_house_sqft, estimated_price))

The estimated price for a house with 2650 squarefeet is $704259.61


# Residual Sum of Squares

Now that we have a model and can make predictions let's evaluate our model using Residual Sum of Squares (RSS). Recall that RSS is the sum of the squares of the residuals and the residuals is just a fancy word for the difference between the predicted output and the true output. 

Complete the following (or write your own) function to compute the RSS of a simple linear regression model given the input_feature, output, intercept and slope:

In [ ]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    # First get the predictions
    predicted_values = get_regression_predictions(input_feature, intercept, slope) # 一樣利用已知 slope w1 & intercept w0, 對輸入的input_feature 求得其估計值
    # then compute the residuals (since we are squaring it doesn't matter which order you subtract)
    residuals =  output - predicted_values
    # square the residuals and add them up
    RSS = (residuals**2).sum()
    return(RSS)

Let's test our get_residual_sum_of_squares function by applying it to the test model where the data lie exactly on a line. Since they lie exactly on a line the residual sum of squares should be zero!

In [ ]:
print (get_residual_sum_of_squares(test_feature, test_output, test_intercept, test_slope)) # should be 0.0

0.0


Now use your function to calculate the RSS on training data from the squarefeet model calculated above.

**Quiz Question: According to this function and the slope and intercept from the squarefeet model What is the RSS for the simple linear regression using squarefeet to predict prices on TRAINING data?**

In [ ]:
rss_prices_on_sqft = get_residual_sum_of_squares(train_data['sqft_living'], train_data['price'], sqft_intercept, sqft_slope)
print ('The RSS of predicting Prices based on Square Feet is : ' + str(rss_prices_on_sqft))

The RSS of predicting Prices based on Square Feet is : 1209822763116208.0


# Predict the squarefeet given price


What if we want to predict the squarefoot given the price? Since we have an equation y = a + b\*x we can solve the function for x. So that if we have the intercept (a) and the slope (b) and the price (y) we can solve for the estimated squarefeet (x).

Complete the following function to compute the inverse regression estimate, i.e. predict the input_feature given the output.

In [ ]:
def inverse_regression_predictions(output, intercept, slope):
    # solve output = intercept + slope*input_feature for input_feature. Use this equation to compute the inverse predictions:
    #input_feature = (output - intercept) / slope
    estimated_feature = (output - intercept) / slope
    return estimated_feature

Now that we have a function to compute the squarefeet given the price from our simple regression model let's see how big we might expect a house that costs $800,000 to be.

**Quiz Question: According to this function and the regression slope and intercept from (3) what is the estimated square-feet for a house costing $800,000?**

In [ ]:
my_house_price = 800000
estimated_squarefeet = inverse_regression_predictions(my_house_price, sqft_intercept, sqft_slope) # Using inverse_regression_predictions to find out
print ("The estimated squarefeet for a house worth $%.2f is %d" % (my_house_price, estimated_squarefeet))

The estimated squarefeet for a house worth $800000.00 is 2987


# New Model: estimate prices from bedrooms

We have made one model for predicting house prices using squarefeet, but there are many other features in the sales SFrame. 
Use your simple linear regression function to estimate the regression parameters from predicting Prices based on number of bedrooms. Use the training data!

In [ ]:
# Estimate the slope and intercept for predicting 'price' based on 'bedrooms'
sqft_intercept2, sqft_slope2 = simple_linear_regression(train_data['bedrooms'], train_data['price'])
print ("Intercept: " + str(sqft_intercept2))
print ("Slope: " + str(sqft_slope2))

Intercept: 126751.84729183262
Slope: 123535.45861796693


# Test your Linear Regression Algorithm

Now we have two models for predicting the price of a house. How do we know which one is better? Calculate the RSS on the TEST data (remember this data wasn't involved in learning the model). Compute the RSS from predicting prices using bedrooms and from predicting prices using squarefeet.

**Quiz Question: Which model (square feet or bedrooms) has lowest RSS on TEST data? Think about why this might be the case.**

In [ ]:
# Compute RSS when using bedrooms on TEST data:

sqft_intercept2, sqft_slope2 = simple_linear_regression(train_data['bedrooms'], train_data['price']) # 利用已經建立好的 function 求得 intercept & slope
rss_prices_on_bedrooms =  get_residual_sum_of_squares(train_data['bedrooms'], train_data['price'], sqft_intercept, sqft_slope)# 利用已經建立好的 function 求得 RSS
print ('The RSS of predicting Prices based on Bedrooms is : ' + str(rss_prices_on_bedrooms))


The RSS of predicting Prices based on Bedrooms is : 8417794200127937.0


In [ ]:
# Compute RSS when using squarefeet on TEST data:
sqft_intercept, sqft_slope = simple_linear_regression(train_data['sqft_living'], train_data['price'])
rss_prices_on_sqft = get_residual_sum_of_squares(train_data['sqft_living'], train_data['price'], sqft_intercept, sqft_slope)
print ('The RSS of predicting Prices based on Square Feet is : ' + str(rss_prices_on_sqft))

The RSS of predicting Prices based on Square Feet is : 1209822763116208.0


In [ ]:
# 上傳.ipynb檔
from matplotlib.cbook import print_cycles
from google.colab import files
upload = files.upload()
for filename in upload.keys():
  print('file"{name}"'.format(name = filename))

In [ ]:
# .ipynb檔轉換成.html檔
!jupyter nbconvert --to html *.ipynb